# Applying constraints and features
This notebook will showcase the usage of constraints and features.

Lets start by creating a simple 3d model of a simply supported beam and a vertical column to be attached at midspan:

In [1]:
from beef import fe
import numpy as np

node_matrix = np.array([[1, 0, 0, 0],
                        [2, 5, 0, 0],
                        [3, 10, 0, 0],
                        [11, 5, 0, 0.5],
                        [12, 5, 0, 5],
                        [13, 5, 0, 10]
                       ])

element_matrix = np.array([[1, 1, 2],
                          [2, 2, 3],
                          [3, 11, 12],
                          [4,12,13]])

supports = [fe.Constraint([3], dofs='all'), fe.Constraint([1], dofs='all')]

# Section definition
E = 210e9
A = 0.1
I_y = 0.05
I_z = 0.05
J = 0.1
section = fe.Section(E, A=A, I_y=I_y, I_z=I_z, J=J)

part = fe.Part(node_matrix, element_matrix, constraints=supports, sections=section)
part.plot(node_labels=True)

Widget(value="<iframe src='http://localhost:59875/index.html?ui=P_0x2176b2586d0_0&reconnect=auto' style='width…

For visibility, the two nodes 11 and 2 are placed a small distance apart. However, in many realistic cases, it would be reasonable to give them the exact same position.

## Connection using constraints

Constraints are defined as fixed links between master nodes and slave nodes. If only one node list is input, the specified DOFs of the nodes are considered fixed to the ground. Currently, no constraint stiffness is supported. To introduce stiffness (different than infinite) in constraints, the the relevant coupled nodes must be modelled apart and a feature placed between the relevant nodes (see next section for an example).

If node 2 and 11 are to be fully fixed, this is done by the following constraint:


In [2]:
# All DOFs to be constrainted, specified using dofs='all' or dofs=[0,1,2,3,4,5]
constraint_beam_to_column = [fe.Constraint([2], [11], dofs='all')]

part = fe.Part(node_matrix, element_matrix, constraints=supports+constraint_beam_to_column, sections=section)

part.plot(constraints_on=['undeformed'], plot_constraints=['relative'], 
          node_labels=True, constraint_opts=dict(color='black'))

Widget(value="<iframe src='http://localhost:59875/index.html?ui=P_0x2176957db40_1&reconnect=auto' style='width…

The constrained part is loaded by a 5MN force on top of the column to verify it has a reasonable behaviour:

In [3]:
force = fe.Force([13], dofs=[1], amplitudes=[5.0e6], t=1)
analysis = fe.Analysis(part, forces = [force])
analysis.run_lin_static()
analysis.eldef.plot(plot_states=['undeformed', 'deformed'], 
                    constraints_on=['undeformed', 'deformed'], plot_constraints=['relative'])

Static analysis:   0%|          | 0/2 [00:00<?, ?it/s]

Widget(value="<iframe src='http://localhost:59875/index.html?ui=P_0x2176fd25870_2&reconnect=auto' style='width…

## Connection using features

Now, lets try to model the same behaviour, but where the connection has a specified stiffness instead of a complete fixation. Then, a feature is needed. The following features are currently supported in beef:

* `Spring`
* `Dashpot` 
* `PointMass` 
* `CustomMatrix` 

It is referred to the documentation (in code or online) for more details. However, all four features are inherited from the main `Feature` class. The `CustomMatrix` feature is the most flexible as it allows the user to directly specify the wanted matrix. The other the features are less flexible and assumes that the same DOFs (global) of the various nodes are connected. For example, the feature `fe.Spring([2,11], [0,1,2], 100)` creates three springs of stiffness 100 N/m between nodes 2 and 11: one along global DOF 0, one along global DOF 1 and one along global DOF 2. The feature`fe.Spring([2], [0,1,2], 100)` does the same, but assumes that the end previously connected to 11 is now instead connected to the ground.

To model the same connection as above using springs, we introduce the following very stiff springs (k = 1000 GN/m or 1000 GNm/rad):

In [4]:
k = 1000e9
dofs = [0,1,2,3,4,5]
node_labels = [2,11]
spring_connection = [fe.Spring(node_labels, dofs, k)]
part = fe.Part(node_matrix, element_matrix, constraints=supports, features=spring_connection, sections=section)

In [5]:
analysis = fe.Analysis(part, forces=[force])
analysis.run_lin_static()

Static analysis:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
analysis.eldef.plot(plot_states=['undeformed', 'deformed'])

Widget(value="<iframe src='http://localhost:59875/index.html?ui=P_0x2177b5212a0_3&reconnect=auto' style='width…

Alternatively, the joint/connection could have different stiffnesses for different DOFs. This can be modelled using a separate `Spring` feature for each stiffness. In this example, the stiffness between the column and the beam in the direction of the force is chosen much lower than the rest (e.g. the moment stiffness):
* Moderate spring: $k_y = 10 MN/m$
* Very stiff springs: $k_i = 1000GN/m$ for $i=x,z,\theta_x, \theta_y, \theta_z$

The following cell show how this is introduced:

In [7]:
k_stiff = 1000e9
k_flexible = 10e6
node_labels = [2,11]

spring_connections = [fe.Spring(node_labels, [1], k_flexible),   # less stiff connection laterally to supported beam
                     fe.Spring(node_labels, [0,2,3,4,5], k_stiff), ]  # very stiff in all other DOFs

part = fe.Part(node_matrix, element_matrix, constraints=supports, features=spring_connections, sections=section)

In [8]:
analysis = fe.Analysis(part, forces=[force])
analysis.run_lin_static()
analysis.eldef.plot(plot_states=['undeformed', 'deformed'])

Static analysis:   0%|          | 0/2 [00:00<?, ?it/s]

Widget(value="<iframe src='http://localhost:59875/index.html?ui=P_0x2176fd26470_4&reconnect=auto' style='width…

Finally, an additional force is added along the beam as well to better illustrate the difference in stiffness:

In [9]:
forces = [fe.Force([12], dofs=[0], amplitudes=[70.0e6], t=1),
         force]

part = fe.Part(node_matrix, element_matrix, constraints=supports, features=spring_connections, sections=section)

analysis = fe.Analysis(part, forces=forces)
analysis.run_lin_static()
analysis.eldef.plot(plot_states=['undeformed', 'deformed'])

Static analysis:   0%|          | 0/2 [00:00<?, ?it/s]

Widget(value="<iframe src='http://localhost:59875/index.html?ui=P_0x2177b5217e0_5&reconnect=auto' style='width…